In [1]:
from ultralytics import YOLO

model = YOLO('../yolo11n-pose.pt')

model.export(
    format='engine',
    half=True,
    imgsz=640,
    workspace=8,
    # batch=8,
    # int8=True,
    # data='coco-pose.yaml',
)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.27 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 12029MiB)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

PyTorch: starting from '../yolo11n-pose.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 56, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 2.4 MB/s eta 0:00:0000:0100:04

requirements: AutoUpdate success ✅ 127.9s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.36...
ONNX: export success ✅ 128.7s, s

'../yolo11n-pose.engine'

# 3x speedup

In [2]:
import cv2
tensorrt_model = YOLO('../yolo11n-pose.engine')
video_path = '../fall_dataset/videos/video_2.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened:
    print("Error could not open video")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = tensorrt_model(frame)
    for result in results:
        frame = result.plot()

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Loading ../yolo11n-pose.engine for TensorRT inference...
[11/03/2024-17:42:35] [TRT] [I] Loaded engine size: 9 MiB
[11/03/2024-17:42:35] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +10, now: CPU 0, GPU 15 (MiB)

0: 640x640 3 persons, 1.4ms
Speed: 1.4ms preprocess, 1.4ms inference, 56.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 2.9ms
Speed: 3.4ms preprocess, 2.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 2.7ms
Speed: 2.6ms preprocess, 2.7ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.9ms
Speed: 1.1ms preprocess, 1.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.4ms
Speed: 1.1ms preprocess, 1.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.4ms
Speed: 1.2ms preprocess, 1.4ms inference, 0.8ms postprocess per image at shape (